In [1]:
import chromadb
from libraries.db import DatabaseProxy

In [2]:
chroma_client = chromadb.PersistentClient("./instance")
collection = chroma_client.create_collection(name="my_collection", metadata={"hnsw:space": "cosine"})

In [3]:
db = DatabaseProxy()

In [4]:
questions = db.get_questions()

In [5]:
documents = []
metadatas = []
ids = []
for question in questions:
    documents.append(question.text)
    metadatas.append({"id": question.id})
    ids.append(str(question.id))

In [6]:
collection.add(ids, documents=documents, metadatas=metadatas)

In [7]:
questions[:5]

[Question(id=1, text='Checking the financial stability of our suppliers is part of our due diligence process. To support us in doing this, please tell us when your company financial year runs to and from.', answer_id=1),
 Question(id=2, text='How many candidates do you have in your database?', answer_id=2),
 Question(id=3, text='How do you select which candidates are placed where?  For instance, if two buyers are looking for the same candidate, how do you ultimately determine which company to place that employee with?', answer_id=3),
 Question(id=4, text='How often do you review and refresh your database of candidates?', answer_id=4),
 Question(id=5, text='What type of background and drug testing do you conduct?  How extensive are they? Who is your service provider for background checking?', answer_id=5)]

In [11]:
def print_results(results):
    for i in range(len(results["ids"][0])):
        print(results["ids"][0][i], round(results["distances"][0][i], 2), results["documents"][0][i])
        question = db.get_question_by_id(results["ids"][0][i])
        print(db.get_answer_by_id(question.answer_id).text)
        print()

In [12]:
results = collection.query(
    query_texts=["How many employees work at your firm?"],
    n_results=3
)
print_results(results)

22 0.44 Provide a brief of company history and number of employees
545 employees

158 0.46 What is your most recent satisfaction score for the employees youÕve placed?
We continually hold ourselves accountable to the satisfaction of the associates we place out on assignment. Our branches are scored weekly on the first day follow up calls to associates recently placed. Additionally, our on-site representatives conduct weekly associate satisfaction surveys on 10% of the population, scored weekly on the completion percentage, to ensure we trend in the right direction with happy associates and a good work environment.

151 0.56 If you were a provider of ours last season what was your turnover rate?  If you did not provide labor to us last season what is your average turnover rate?
We were not a provider last season
YTD Turnover % 
CA Market - 44%
TX Market - 65%
NJ Market - 0% (new market)



In [13]:
results = collection.query(
    query_texts=["How often do you hire new employees?"],
    n_results=3
)
print_results(results)

152 0.48 What action or steps do you implement to retain employees throughout the season?
Ex 1: We perform extensive screening for candidates that best match the job descriptions and are comfortable with the expectations at each site. We take care of all of our associates and over-communicate to ensure successful placement.                                                                                                                                                           
Ex 2: Employee Recognition
We are firm believers in recognizing and rewarding good behavior and have met with much success using some of these employee appreciation efforts 
¥	Employee of the Month Ð working together with supervisory staff in identifying deserving associates each month.
¥	Raffle Employee Attendance Program.  Anyone with 100% attendance with 40 hours plus in a week, raffle items (gas cards, gift cards, CASH). 
¥	Employee ÒGoalÓ Lunch or Raffle:  When an employee or ÒTeamÓ achieves a set goal or exc

In [14]:
results = collection.query(
    query_texts=["How will WalMart be involved in the hiring and interview process?"],
    n_results=3
)
print_results(results)

7 0.57 Please advise how Michaels will be utilized in the hiring and onboarding process.  For example, who provides the approvals and onboarding training?
We can accommodate to the needs of each location whether that be an onboarding process conducted in our local branch location or an on-site onboarding process. Our goal is to work with the local teams at Michaels to ensure on-boarding/training is successful and candidates start with the goal of being successful when placed.

3 0.58 How do you select which candidates are placed where?  For instance, if two buyers are looking for the same candidate, how do you ultimately determine which company to place that employee with?
Our specialized role known as the client specialist is a dedicated resource to our customers. Their primary role is to intake orders evaluate priority and review with candidates and customers opportunities presented. In the instance of competing orders candidates would be vetted for each role to ensure they are up to